In [2]:
import os
import time
import math
import numpy as np
from IPython.utils import io
import pandas as pd
from tqdm import tqdm
from dataclasses import dataclass
from typing import List, Union 
import matplotlib.pyplot as plt
import matplotlib
from backtesting import Backtest, Strategy
import multiprocessing as mp
from backtesting.lib import crossover
import backtesting
import sambo
import joblib
import ta
import torch
import torch.nn as nn
from PERMUTATIONS import get_permutation
from strategy import ARSIstrat
import MLP_WFSIG_functions as MLP

/Users/matteogiori/trycloneinsider/TradingBot/.venv313/lib/python3.13/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

                       Open    High      Low    Close  Volume
time                                                         
2018-01-02 14:30:00  156.50  157.88  156.170  157.840  875636
2018-01-02 15:30:00  157.85  158.15  157.630  158.000  424637
2018-01-02 16:30:00  158.01  158.06  157.895  157.990  161584
2018-01-02 17:30:00  157.98  158.10  157.900  158.080  131613
2018-01-02 18:30:00  158.08  158.33  158.070  158.265  251044
                        Open    High     Low    Close   Volume
time                                                          
2018-02-01 14:30:00  168.120  169.30  168.10  168.860   778080
2018-02-01 15:30:00  168.890  169.56  168.77  169.000   369077
2018-02-01 16:30:00  169.030  169.56  169.01  169.350   308644
2018-02-01 17:30:00  169.350  169.72  169.33  169.610   233919
2018-02-01 18:30:00  169.610  169.62  168.15  168.500   536976
2018-02-01 19:30:00  168.530  168.84  168.16  168.180   472291
2018-02-01 20:30:00  168.180  168.28  167.43  167.970   881835

In [3]:
csv_path = 'BATS_QQQ, 60_a45be.csv'

full_df = MLP.load_data(csv_path, start='2018-01-01', end_excl='2024-01-01')
df_train = full_df.loc['2018-01-01':'2021-12-31']
df_test = full_df.loc['2023-01-01':'2023-12-31']

In [4]:
MIN_GAP_FS = 3
MIN_GAP_SV = 7
def _order_ok(p):
    return (
        (p.n_slow  - p.n_fast  >= MIN_GAP_FS) and
        (p.n_vslow - p.n_slow  >= MIN_GAP_SV) and
        (p.n_fast  < p.n_slow  < p.n_vslow)   and
        (p.sig_len > 0) and (p.sl_pct > 0)
    )

In [ ]:
n_permutations = 100
perm_better_count1 = 0
permuted_SRs = []
print("In-Sample MCPT")

SL_GRID = [round(x, 3) for x in np.arange(0.001, 0.020 + 0.0005, 0.001)]

param_space = dict(
    n_fast  = list(range(5, 26, 1)),
    n_slow  = list(range(10, 56, 1)),
    n_vslow = list(range(30, 121, 1)),
    sig_len = list(range(5, 31, 1)),
    sl_pct  = SL_GRID,                         # <<< discrete in 0.001 steps
)

local_df_train = df_train.copy()
local_bt_train = Backtest(local_df_train, ARSIstrat, cash=1_000_000, commission=0.0, spread=0.0001, finalize_trades=True) 

with io.capture_output():
    stats1, _ = local_bt_train.optimize(
        **param_space,
        maximize='Sharpe Ratio',
        return_heatmap=True,
        constraint=_order_ok,
        method='sambo',
        max_tries=2500,
        random_state=42
    )

best = stats1._strategy
best_params = {
    'n_fast' : int(best.n_fast),
    'n_slow' : int(best.n_slow),
    'n_vslow' : int(best.n_vslow),
    'sig_len' : int(best.sig_len),
    'sl_pct' : int(best.sl_pct)
    }

print(best_params)


print(stats1[['Sharpe Ratio', 'Return [%]', '# Trades']])

local_SR_OPTI_0 = stats1['Sharpe Ratio']

for perm_i in tqdm(range(1, n_permutations+1)):

    train_perm = get_permutation(local_df_train)

    train_perm_bt = Backtest(train_perm, ARSIstrat, cash=1_000_000, commission=0.0, spread=0.0001, finalize_trades=True) 
    with io.capture_output():
        stats2, _ = train_perm_bt.optimize(
            **param_space,
            maximize='Sharpe Ratio',
            return_heatmap=True,
            constraint=_order_ok,
            method='sambo',
            max_tries=2500,
            random_state=42
        )

    
    
    best_perm_SR = stats2['Sharpe Ratio']

    
    if best_perm_SR >= local_SR_OPTI_0:
        perm_better_count1 += 1

    permuted_SRs.append(best_perm_SR)

insample_mcpt_pval = perm_better_count1 / n_permutations




print(f"In-sample MCPT P-Value: {insample_mcpt_pval}")
plt.style.use('dark_background')
pd.Series(permuted_SRs).hist(color='blue', label='Permutations')
plt.axvline(local_SR_OPTI_0, color='red', label='Real')
plt.xlabel("Profit Factor")
plt.title(f"In-sample MCPT. P-Value: {insample_mcpt_pval}")
plt.grid(False)
plt.legend()
plt.show()

In-Sample MCPT
